In [65]:
import numpy
import librosa
import matplotlib.pyplot as plot
import sklearn
from sklearn.model_selection import train_test_split
import random
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from keras.utils import to_categorical
import pandas as pd
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn import tree
import graphviz 
import pandas as pd
import pickle
%matplotlib inline

csv = pd.read_csv('ratings2_interrator_averaged.csv', sep = ';') #rating2combined
#print(csv)


clip_length_seconds = 120
downsampling_rate = 1
count_videos = 16
count_clips = 5
split_ratio = 0.90
count_models = 1

X = []
Y = []

for i in range(1, 1 + count_videos):
    filename = 'vid' + str(i) + '.wav'    
    wave, sr = librosa.load(filename, mono=True, sr=None)
    samplesi = csv.loc[csv['video_id'] == i]
    
    for j in range(0, count_clips):
        start = j * clip_length_seconds * sr
        end = (j + 1) * clip_length_seconds * sr
        
        wave_clip = wave[start:end]
        #print('Clip shape for j = ' + str(j) + ': ' + str(wave_clip.shape))

        wave_downsampled = wave_clip[::downsampling_rate]
        #print('Downsample shape for j = ' + str(j) + ': ' + str(wave_downsampled.shape))

        mfcc = librosa.feature.mfcc(wave_downsampled, sr)
        #print('Padded mfcc shape for j = ' + str(j) + ': ' + str(mfcc.shape))
        X.append(mfcc)
        samplesj = samplesi.loc[samplesi['section'] == j + 1]
        Y.append(samplesj.drop(['video_id', 'section'], axis=1).values.flatten())
            

for label_set in Y:
    label_set[:] =  [1 if label==2 else label for label in label_set]
    label_set[:] =  [2 if label==3 else label for label in label_set]
    label_set[:] =  [3 if label==4 else label for label in label_set]
    label_set[:] =  [3 if label==5 else label for label in label_set]

X = numpy.array(X)
Y = numpy.array(Y)

#Y = Y[:,2:]

for model_type in range(0, Y.shape[1]):
    tot_acc_svn = 0
    tot_acc_dt = 0
    random_state_list = [98, 91, 8, 53]
    index=0;
    
    y = Y[:, model_type]
    
    #print(y)

    for mdl in range(0, count_models):
        #random_state = random.randint(1,101)
        #print('Random state: ' + str(random_state))
        
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = (1 - split_ratio), random_state = random_state_list[index], shuffle = True)
        print(X_train.shape)
        # Support vector machine (baseline)
        X_train_t = [i.flatten() for i in X_train]
        X_test_t = [i.flatten() for i in X_test]
        print(len(X_test_t))
        svclassifier = SVC(kernel='linear')  #kernel='rbf', C = 0.1, gamma=1
        svclassifier.fit(X_train_t, y_train)
        y_pred = svclassifier.predict(X_test_t)  
        
        tot_acc_svn += accuracy_score(y_test,y_pred)
    i += 1
    print("The average accuracy of SVN:" + str(tot_acc_svn / count_models))
    #print("The average accuracy of DTree:" + str(tot_acc_dt / count_models))
    
# save the model to disk
    filename = str(model_type) + '.sav'
    pickle.dump(svclassifier, open(filename, 'wb'))



(72, 20, 10336)
8
The average accuracy of SVN:0.625
(72, 20, 10336)
8
The average accuracy of SVN:0.5
(72, 20, 10336)
8
The average accuracy of SVN:0.375
(72, 20, 10336)
8
The average accuracy of SVN:0.125


In [78]:
import numpy
import librosa
import json

count_clips = 5
# load the model from disk
models = ['0.sav','1.sav','2.sav', '3.sav']
loaded_model = []
index = 0
results= []
X = []
for filename in models:
    print(index)
    loaded_model.append(pickle.load(open(filename, 'rb')))
    index += 1

filename = 'record.wav'
wave, sr = librosa.load(filename, mono=True, sr=None)

for j in range(0, count_clips):
    start = j * clip_length_seconds * sr
    end = (j + 1) * clip_length_seconds * sr    
    wave_clip = wave[start:end]
    #print('Clip shape for j = ' + str(j) + ': ' + str(wave_clip.shape))
    wave_downsampled = wave_clip[::downsampling_rate]
    #print('Downsample shape for j = ' + str(j) + ': ' + str(wave_downsampled.shape))
    mfcc = librosa.feature.mfcc(wave_downsampled, sr)
    X.append(mfcc)
    
X = numpy.array(X)

index_i = 0
for models in loaded_model:
    x_test = [x.flatten() for x in X]
    result = loaded_model[index_i].predict(x_test)
    results.append(result)
    index_i+=1
        
print(type(results))
#data = {}
#data['key'] = 'value'
#json_data = json.dumps(data)

Loading Models
0
1
2
3
(5, 20, 10336)
<type 'list'>
